In [ ]:
# Importing tensorflow to python!
import tensorflow as tf

In [ ]:
# Defining a node of a constant in tensorflow 
# tf.constant returns a tf.Tensor and defines a tf.Operation in the graph 
a = tf.constant(42,dtype=tf.int32,name='my_a',shape=(1,))
b = tf.constant(42,dtype=tf.int32,name='my_b',shape=(1,))

In [ ]:
# Defining a sum in tensorflow
# will add a tf.Operation to the graph
c = tf.add(a,b) 
# or c = a + b, tensorflow accepts a range of arithmetic and logical ops

In [ ]:
# but what is c? c is still a tensor, tf.add() returns a tensor and
# adds an operation to the graph
c

In [ ]:
# we need to create a session in order to run a specific node in the
# graph. Furthermore, we have been adding nodes to the "default" graph,
# created when importing the tensorflow module, but we could've created
# a different one

In [ ]:
# Create a session object to run nodes in the graph
sess = tf.Session()

In [ ]:
# Now we may run any node in the graph by using the tf.Session.run()
# on any node we wish. 
# The nodes are the tensors and the operations the edges of the graph!

In [ ]:
print('My value of a is {}, b is {} and the sum c is {}'.format(sess.run(a),sess.run(b),sess.run(c)))

In [ ]:
# We may close a session with sess.close() which will give us an error when trying to run a node
sess.close()
sess.run(a)

In [ ]:
# We can open a different session with tf.Session() again.
sess = tf.Session()

In [ ]:
# However, we need to dive deeper in the tensorflow structure. Tensorflow was designed to have superior computational
# efficienty at a cost of having to define a computational graph and associated variables properly
# Let's see how many tensors and operations we have so far

In [ ]:
# Get all the tensors currently in our default graph
tf.contrib.graph_editor.get_tensors(tf.get_default_graph())

In [ ]:
# And visualize each tf.Operation added to the graph
sess.graph.get_operations()

In [ ]:
# We may notice that we have the same amount of tf.Tensors and tf.Operations.
# The fact is, when creating anything in TensorFlow we always need a space in memory to physically have each tensor
# And we also need a way to compute and feed each tensor onto other nodes, by defining the operations (edges)
# Let's save our graph and visualize it on tensorboard!
tf.train.write_graph(tf.get_default_graph(), 'folder', 'our_graph.pb')

In [ ]:
# However, we may reset the graph and all the nodes we created, since jupyter notebook is an interactive
# python shell, remenber to reset the graph from time to time :>
tf.reset_default_graph()

In [ ]:
# And visualize there are no tensors or operations in our graph
tf.contrib.graph_editor.get_tensors(tf.get_default_graph())

In [ ]:
# One of the important things to be adressed. Everytime we call a tf function
# we create a tensor (and consequently an operation), this leads to subtle memory leaks.
for i in range(5):
    a = tf.constant(42,dtype=tf.int32,name='my_a',shape=(1,))
tf.contrib.graph_editor.get_tensors(tf.get_default_graph())

In [ ]:
tf.reset_default_graph()
# When desiging graphs (such as neural networks) it is a good practice to define all the nodes at start
# and running only the nodes in our code
a = tf.constant(42,dtype=tf.int32,name='my_a',shape=(1,))
for i in range(5):
    print(sess.run(a)) 
tf.contrib.graph_editor.get_tensors(tf.get_default_graph())

In [ ]:
# Handy Reset
tf.reset_default_graph()

In [ ]:
# We may create tf.Variables, which will hold a variable Tensor during computations
# Most nodes will be composed of tf.Variables.
# There are two ways to define new variables in TensorFlow, by creating a 
# tf.Variable object or by calling tf.get_variable. 
# Calling tf.get_variable with a new name results in creating a new variable, 
# but if a variable with the same name exists it 
# will raise a ValueError exception, telling us that re-declaring 
# a variable is not allowed.
d = tf.get_variable('my_d',[1,2,3]) #Here we already defined the shape 1-D array
e = tf.get_variable('my_e', shape=(18,)) #Here the shape an 1-D array
f = tf.get_variable('my_f', dtype=tf.int32, initializer=tf.constant([23, 42])) 
#Here we defined the type and the initializer which happens to be a constant tf.constant, the shape is defined
tf.contrib.graph_editor.get_tensors(tf.get_default_graph())
# Why are there so many tensors? the second argument is the shape. The default initializer of variables
# is a random uniform distribution. Each Tensor of this Initializer is also created under the name of each variable
# Notice the Tensors Assign and read, serve to feed and fetch values from the Tensor 'my_variable0':0
# In the 'my_f' variable we only have 4 tensors, the initializer, which is a tf.constant, the "de facto" variable
# 'my_f:0' and 2 other tensors to read and assing values to 'my_f:0'

In [ ]:
# Lets get the session again, instead of creating a new tf.Session, this is handy on long, complex programs.
# If there isn't any tf.Session available, it will return 'None', so we need to create another session.
# sess = tf.Session()
sess = tf.get_default_session()
# Run the node f and print it
sess.run(f)
# Why did we got an error? the tf.initalizer is a method, and has not been run, therefore
# there are no values to be retrieved when we run the node. We have two options: 
# 1) Either run a particular variable initializer with sess.run(f.initializer)
# 2) Or run the tf.global_variables_initializer() which will initialize all the nodes created in the graph thus far

In [7]:
sess.run(tf.global_variables_initializer())
sess.run(f)
# Every node (tensor) when runned, return an numpy array

array([23, 42], dtype=int32)

In [ ]:
sess.graph.get_tensor_by_name('my_a:0')

tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)

g = tf.get_variable('my_d') #Here we use the same tensor name 'my_d' created in d to perform
# a different computation with the same weights on another part of the program

When we run a particular node,  

sess.run(a)



session.run(b)

sess.close()